----
SQL: Advanced Querying
---

<img src="https://s3-us-west-2.amazonaws.com/dsci/6007/assets/sql_joke.jpg" style="width: 400px;"/>

---
By the end of this session, you should be able to:
----

- See the connection between many Python concepts and SQL
    - Aliasing
    - Missing values
- Model relationship among tables using JOIN
- Write queries using INNNER JOIN, LEFT OUTER JOIN, and FULL JOIN

---
Aliasing
---

remeber: `import numpy as np`

SQL has that too!

What will this query give us?

```sql
SELECT count(distinct(artist))
  FROM tutorial.billboard_top_100_year_end
```

<details><summary>
Click here for solution
</summary>
Number of artists in the table
</details>

<br>
<br>
<br>
<br>

```sql
SELECT count(distinct(artist)) as "Number of Artists"
  FROM tutorial.billboard_top_100_year_end
  
or

SELECT count(distinct(artist)) "Number of Artists"
  FROM tutorial.billboard_top_100_year_end
```

https://modeanalytics.com/editor/brianspiering/reports/293bd6004b03

---

Materialized view is the results of a query, aka results table

Remeber we are being 'functional' by creating views on immutable data.

---
3 value logic
----

1. TRUE
2. FALSE
3. NULL

`NULL` It is not a value -  It is a “place holder” for a value. 

The original intent of NULL in SQL was to represent missing data in a database, i.e. the assumption that an actual value exists, but that the value is not currently recorded in the database.

[Source](https://en.wikipedia.org/wiki/Three-valued_logic#Application_in_SQL)

---
Check for understanding
---

What is the Python keyword for NULL?

<details><summary>
Click here for solution...
</summary>
[`None`](https://docs.python.org/2/library/constants.html)
</details>


Which is the correct syntax?

`... where department = NULL`  
or  
`... where department is NULL`

Why?

<details><summary>
Click here for solution...
</summary>
`... where department is NULL`
<br>
You can’t perform arithmetic on null values.
</details>

<details><summary>
Is empty string the same `NULL`? Why or why not?
</summary>
__No__ Empty string is a TRUE string. NULL is absence of value.
</details>

---
Exploring missing values
----

```sql
SELECT count(*) 
  FROM tutorial.billboard_top_100_year_end --=>6476

SELECT count(*) 
  FROM tutorial.billboard_top_100_year_end
  WHERE artist is null --=> 9

SELECT count(*) 
  FROM tutorial.billboard_top_100_year_end
  WHERE artist is not null --=> 6467 
```

----
Modeling Relations in RDBMS
----

A **primary key** is a column in a table that uniquely identifies that entry. 

No two rows in the same table can share a value for a column specified as primary
key. The primary key column is most often `id`.

Here's an example of what this users table looks like:

```
 id | name  | age |     city      | state
----+-------+-----+---------------+-------
  1 | john  |  25 | San Francisco | CA
  2 | becky |  30 | NYC           | NY
  3 | sarah |  20 | Denver        | CO
... | 
```

Part of the power of relational databases are their ability to model relations
in data. The way they do so is through the use of **foreign keys**.
A foreign key is a column that references some other entry in the database.
That foreign entry could be in the same table or in some other table. Foreign
keys are how relations are modeled in relational databases.

For example, let's say there is another table that contains data for each visit
to our website. Each time a user visits the site, a row is created and inserted
into the `visits` table. We'd like to maintain some data that indicates which
visit is associated with which user (so that we could later, for example, find
the users who visited our site the most). Each visit then will have a `user_id`
that can connect it to a user in the `users` table.

Here's the definition of the `visits` table:

```sql
CREATE TABLE VISITS (
  id INTEGER PRIMARY KEY
,  created_at TIMESTAMP
,  customer_id INTEGER REFERENCES customers(id) );
```

Here's an example of what this users table looks like:

```
 id |     created_at      | customer_id
----+---------------------+-------------
  1 | 2015-06-20 00:00:00 |           1
  2 | 2015-07-30 00:00:00 |           1
  3 | 2015-06-20 00:00:00 |           3
  4 | 2015-04-09 00:00:00 |           1
  5 | 2015-03-09 00:00:00 |           2
... | 
```

Here we specify not only that the `visits` table has a column called `customer_id`, but that the column references the `id` column in the `customers` table. PostgreSQL will treat this as a constraint and ensure that new visits have a `customer_id` value that references an actual user in the database.

### Aggregations

SQL allows you to aggregate your data set based on common keys. 

To see the number of visits from each customer_id we would query:

```sql
SELECT customer_id, COUNT(*)
FROM visits
GROUP BY customer_id
```

```
 customer_id | count
-------------+-------
           1 |     3
           3 |     1
           2 |     1
```

**Note:** The GROUP BY clause tells SQL what common factor we'd like to use to aggregate the data.
The COUNT aggregate function tells SQL how we'd like to aggregate.


-----
Relationships Types
-----

1. One-to-one
2. One-to-many/many-to-one
3. Many-to-many


#### One-to-one

For a one-to-one relationship, place the foreign key on either side of the
relationship.

**Example: Customers and Licenses**
Licenses is a table that holds the _most recent_ driver's license number for a customer,
because customers can only have 1 driver's license at time, it will be a one-to-one relationship.

```sql
CREATE TABLE LICENSES (
  id INTEGER PRIMARY KEY
, state VARCHAR(2)
, number VARCHAR(20)
, uploaded_at TIMESTAMP
, customer_id INTEGER REFERENCES customers(id)
, UNIQUE(state, number))
```
```
SELECT * FROM licenses;
 id | state |   number   |     uploaded_at     | customer_id
----+-------+------------+---------------------+-------------
  1 | CO    | DL19480284 | 2013-04-18 00:00:00 |           3
  2 | CA    | DL19852984 | 2014-05-12 00:00:00 |           1
```

To find the license for a customer:

```sql
SELECT *
FROM licenses
WHERE customer_id=?
```
**Notice** we also used a UNIQUE constraint as the last part of our CREATE TABLE statement.
This is part of the power of SQL.  Because State and Driver's Licenses numbers should be unique,
we can help limit data input errors by placing a UNIQUE constraint.

#### One-to-many / many-to-one

For a one-to-many/many-to-one relationship (they are inverses of each other),
place the foreign key on the many side of the relationship.

**Example: Customer and Visits**

To find the visits for a customer:

```sql
SELECT *
FROM visits
WHERE customer_id=?
```

To find details on the customer for that visit:

```sql
SELECT *
FROM customers
WHERE id=?
LIMIT 1
```

#### Many-to-many

For a many-to-many relationship, create a table that contains two foreign keys,
one to each side of the relationship. This intermediate table is often referred
to as a **JOIN table**.

**Example 1: Customers and Products**

Here is our products table.  It lists all the products in the inventory.

```sql
CREATE TABLE PRODUCTS (
  id INTEGER PRIMARY KEY
, name VARCHAR(50)
, price FLOAT
  );
```
```
id |    name     | price
----+-------------+-------
  1 | soccer ball |  20.5
  2 | iPod        |   200
  3 | headphones  |    50
```

How do we know which customer purchased which product?  Here is our **JOIN** table.

```sql
CREATE TABLE PURCHASES (
    id INTEGER PRIMARY KEY
,   customer_id INTEGER REFERENCES customers(id)
,   product_id INTEGER REFERENCES products(id)
,   date TIMESTAMP
,   quantity INTEGER );
```
```
 id | customer_id | product_id |        date         | quantity
----+-------------+------------+---------------------+----------
  1 |           1 |          2 | 2015-07-30 00:00:00 |        2
  2 |           2 |          3 | 2015-06-20 00:00:00 |        3
  3 |           1 |          3 | 2015-04-09 00:00:00 |        1
```

**Note:**  In the customers and products tables, there are PRIMARY KEY constraints on the IDs.
This ensures there is only 1 record for each customer and product.  We then used those keys to 
place FOREIGN KEY constraints on the customer_id and product_id in purchases.  This ensures our
data will only contain purchases by customers already in the database for products already in the
database.  Any other scenario indicates our data is not correct.

----
JOINs
----

The power of RDBMS comes from using relationships. The primary way to understand relationships is the `JOIN` clause. It does so by making use of foreign keys. 

Every `JOIN` clause has two segments: 

1. Specifying the table to join
2. Specifying the columns to match up (usually a primary key from one side matched up with a foreign key on another)

There are many different kinds of `JOIN`s: For now, we'll start with `INNER JOIN`.

Here's an example of a simple `INNER JOIN`:

```sql
SELECT customers.name, visits.created_at
FROM visits
INNER JOIN customers
  ON customers.id = visits.customer_id
```

Each visit has a `customer_id` that corresponds to the `id` column in the customers table. In SQL, you specify the correspondence in the `ON` segment of the `JOIN` clause. 

**For each match between `customers.id` and `visits.customer_id` that is found, a row is inserted into the result set.** That means that if a customer has visited the site multiple times, their information will be in the result set multiple times.

For example, the result may look like this:

```
 name  |     created_at
-------+---------------------
 john  | 2015-06-20 00:00:00
 john  | 2015-07-30 00:00:00
 sarah | 2015-06-20 00:00:00
 john  | 2015-04-09 00:00:00
 becky | 2015-03-09 00:00:00
```

Note how some customers show up multiple times in the result set. This make sense given that some customers visted the site multiple times. 

---
All `JOIN` types
---

![](images/SQL_Joins.svg.png)

The various `JOIN`s specify how to deal with different circumstances regarding the primary and foreign key matchings. 

The most common joins:

- `INNER JOIN`s discard any entries that do not have a match between the keys specified in the `ON` clause. 
- `LEFT OUTER JOIN` keeps all the entries in the left table regardless of whether a match is found in the right table. 
- `FULL OUTER JOIN` will keep the rows of both tables no matter what with `NULL` values for ones that don't have matches.

---
Check for understanding
---

<details><summary>
What is the set terminology for INNNER JOIN?
</summary>
Intersection
</details>
<br>
<details><summary>
What is the set terminology for FULL OUTER JOIN?
</summary>
UNION
</details>

---
Check for understanding
---

| tab1.id |
|:-------:|
| 12 | 
| 14 |
| 10 |
| 11 |


| tab2.id |
|:-------:|
| 13 | 
| 15 |
| 11 |
| 12 |

By hand, figure out the result tables / materialized view for:

- Inner join
- Left outer join where tab1 is left and tab2 is right
- Full outer join

<details><summary>
Click here for solution...
</summary>
<img src="images/sql-joins.gif" style="width: 400px;"/>
</details>

---
More complex joins
----

Let's say for an online sales company you have the following tables:

```sql
customers
    name
    id

products
    name
    id
    price

purchases
    customer_id
    product_id
    date
    quantity
```

### Inner Join

If you want to get all the purchases but you want to include the customer name and the product name, you would use a stardard join (this is also called an *inner join*):

```sql
SELECT customers.name AS cust_name, 
  products.name AS prod_name, 
  date, 
  quantity
FROM purchases
JOIN products ON products.id=purchases.product_id
JOIN customers ON customers.id=purchases.customer_id;
```

### Outer Join

Let's say there's a mistake in our database and some products are in the purchase table but not in the products table. With the above query, they would just end up being excluded. An inner join only includes entries which are in both tables.

We can insure they are included in our result by using an *outer join*.

```sql
SELECT customers.name AS cust_name, products.name AS prod_name, date, quantity
FROM purchases
LEFT OUTER JOIN products ON products.id=purchases.product_id
JOIN customers ON customers.id=purchases.customer_id;
```

The LEFT means that everything from the first table will be included even if there isn't a matching entry in the second table. This will include those missing products, but their names will be `NULL`.

If you'd like to fill in those null values with something, maybe in this case just the product id, you can use the `COALESCE` function (in some implementations of SQL there is an `IFNULL` function).

Note that since the product id is an integer, we have to convert it to a string using the `CAST` function.

```sql
SELECT
    customers.name AS cust_name,
    COALESCE(products.name, CAST(purchases.product_id AS VARCHAR)) AS prod_name,
    date,
    quantity
FROM purchases
LEFT OUTER JOIN products ON products.id=purchases.product_id
JOIN customers ON customers.id=purchases.customer_id;
```

Something that comes up a lot is finding these products which aren't in the products table. We can do this using an outer join:

```sql
SELECT purchases.product_id
FROM purchases
LEFT OUTER JOIN products
ON purchases.product_id=products.id
WHERE products.id IS NULL;
```

---
Subquieres
----

<img src="http://i.imgur.com/EQb5f29.jpg" style="width: 400px;"/>

There is a MAX function which gives you the maximum value, but often you want the entry associated with it. Let's say you want to find the most expensive item. The way to do this is to find the maximum price and then find all the elements whose price is that value.

__Note__: For this we will assume that there's no missing data in our tables.


This requires you to do a nested SELECT statement.

```sql
SELECT *
FROM products
WHERE price=(SELECT MAX(price) FROM products); 
```

Or maybe you want the product which has generated the most profit. Sometimes it's helpful to, instead of having multiple nested selects, creating a temporary table as an intermediary step.

```sql
CREATE TABLE profits AS
SELECT products.name, products.id, SUM(purchases.quantity * products.price) AS profit
FROM products
JOIN purchases
ON products.id=purchases.product_id
GROUP BY products.name, products.id;

SELECT name, id
FROM profits
WHERE profit=(SELECT MAX(profit) FROM profits);
```

You can also do this with a nested query without creating the intermediary table:

```sql
SELECT name, id
FROM profits
WHERE profit=(SELECT MAX(profit)
              FROM (SELECT products.name,
                           products.id,
                           SUM(purchases.quantity * products.price) AS profit
                    FROM products
                    JOIN purchases
                    ON products.id=purchases.product_id
                    GROUP BY products.name, products.id) tmp);
```

Or using a with clause:

```sql
WITH p AS (
SELECT products.name, products.id, SUM(purchases.quantity * products.price) AS profit
FROM products
JOIN purchases
ON products.id=purchases.product_id
GROUP BY products.name, products.id)

SELECT name, id
FROM p
WHERE profit=(SELECT MAX(profit) FROM p);
```

---
OYO
---

- [Variables](http://stackoverflow.com/questions/766657/how-do-you-use-variables-in-a-simple-postgresql-script)
- CASE statements, aka `if-then`
- LIKE keyword, aka regex

---
Keyword Cheatsheet
---

* Return values
  * SELECT
  * DISTINCT
* Tables and rows
  * FROM
  * JOIN
    * INNER
    * LEFT, RIGHT
    * FULL
* Filtering
  * WHERE
  * =, !=, >, <, >=, <=
  * AND, OR
  * IN, NOT IN
  * LIKE
  * IS NULL, IS NOT NULL
  * LIMIT
  * BETWEEN
* Aggregating
  * GROUP BY
  * COUNT
  * MAX, MIN
  * SUM
  * AVG
  * HAVING
* Ordering
  * ORDER BY
* Aliasing
  * AS


---
Summary
----

- SQL has the ability to alias materialized views
- SQL handles missing values with NULL
- Table relations are defined with primary and foreign keys
- Table relations have common design patterns:
    - One-to-one
    - One-to-many
    - Many-to-one
- INNNER JOIN returns only rows common to all tables
- LEFT OUTER JOIN returns all rows in the "left table" And have access to columns in "right table"
- FULL JOIN returns all rows in all tables
- Subquieres allow for more complex manipulation

<br>
<br> 
<br>

----